# ResNet

This software uses cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline

See MIT License in https://github.com/cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline README.md

Wang, Z., Yan, W. and Oates, T. (2017) ‘Time series classification from scratch with deep neural networks: A strong baseline’, 2017 International Joint Conference on Neural Networks (IJCNN), pp. 1578–1585 Online.


In [22]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-


import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, Activation
from tensorflow.keras.initializers import RandomUniform
from tensorflow.keras import utils
from tensorflow.keras.callbacks import ReduceLROnPlateau


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold, RepeatedKFold

import os
import pathlib
import time
from datetime import datetime

np.random.seed(813306)

# User inputs ##

flist = ['GunPoint'] # List dataset directory names.
batch_size = 64 # Wang: int(min(x_train.shape[0]/10, 16)) 
nb_epochs = 1500 # Wang: 1500
tensorboard = True # Set to True to write logs for use by TensorBoard
# Directories
fdir = '../data'  
logs_dir = '../logs'
tensorboard_dir = '../logs/tensorboard'
timestamp = '{:%Y-%m-%dT%H:%M}'.format(datetime.now())
logs_dir = logs_dir +'/' + timestamp
tensorboard_dir = tensorboard_dir +'/' + timestamp

def build_resnet(input_shape, n_feature_maps, nb_classes):
    print ('build conv_x')
    x = Input(shape=(input_shape))
    conv_x = keras.layers.BatchNormalization()(x)
    conv_x = keras.layers.Conv2D(n_feature_maps, 8, 1, padding='same')(conv_x)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
     
    print ('build conv_y')
    conv_y = keras.layers.Conv2D(n_feature_maps, 5, 1, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv2D(n_feature_maps, 3, 1, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)
     
    is_expand_channels = not (input_shape[-1] == n_feature_maps)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps, 1, 1,padding='same')(x)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x)
    print ('Merging skip connection')
    y = keras.layers.add([shortcut_y, conv_z])
    y = Activation('relu')(y)
     
    print ('build conv_x')
    x1 = y
    conv_x = keras.layers.Conv2D(n_feature_maps*2, 8, 1, padding='same')(x1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
     
    print ('build conv_y')
    conv_y = keras.layers.Conv2D(n_feature_maps*2, 5, 1, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv2D(n_feature_maps*2, 3, 1, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)
     
    is_expand_channels = not (input_shape[-1] == n_feature_maps*2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps*2, 1, 1,padding='same')(x1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x1)
    print ('Merging skip connection')
    y = keras.layers.add([shortcut_y, conv_z])
    y = Activation('relu')(y)
     
    print ('build conv_x')
    x1 = y
    conv_x = keras.layers.Conv2D(n_feature_maps*2, 8, 1, padding='same')(x1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = Activation('relu')(conv_x)
     
    print ('build conv_y')
    conv_y = keras.layers.Conv2D(n_feature_maps*2, 5, 1, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = Activation('relu')(conv_y)
     
    print ('build conv_z')
    conv_z = keras.layers.Conv2D(n_feature_maps*2, 3, 1, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    is_expand_channels = not (input_shape[-1] == n_feature_maps*2)
    if is_expand_channels:
        shortcut_y = keras.layers.Conv2D(n_feature_maps*2, 1, 1,padding='same')(x1)
        shortcut_y = keras.layers.BatchNormalization()(shortcut_y)
    else:
        shortcut_y = keras.layers.BatchNormalization()(x1)
    print ('Merging skip connection')
    y = keras.layers.add([shortcut_y, conv_z])
    y = Activation('relu')(y)
     
    full = keras.layers.GlobalAveragePooling2D()(y)   
    out = Dense(nb_classes, activation='softmax')(full)
    print ('        -- model was built.')
    return x, out
 
       
def readucr(filename):
    data = np.loadtxt(filename)
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
   

for each in flist:
    fname = each
    x_train, y_train = readucr(fdir+'/'+fname+'/'+fname+'_TRAIN.txt')
    x_test, y_test = readucr(fdir+'/'+fname+'/'+fname+'_TEST.txt')
    nb_classes = len(np.unique(y_test))
     
    y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
    y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
     
     
    Y_train = utils.to_categorical(y_train, nb_classes)
    Y_test = utils.to_categorical(y_test, nb_classes)
     
    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean)/(x_train_std)
      
    x_test = (x_test - x_train_mean)/(x_train_std)
    x_train = x_train.reshape(x_train.shape + (1,1,))
    x_test = x_test.reshape(x_test.shape + (1,1,))
     
     
    x , y = build_resnet(x_train.shape[1:], 64, nb_classes)
    model = Model(x, y)
    #print(model.summary())
    
    optimizer = keras.optimizers.Adam()
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
      
    reduce_lr = ReduceLROnPlateau(monitor='loss', factor=0.5,
                      patience=50, min_lr=0.0001) 
    callbacks = [reduce_lr]
    if tensorboard:
        tb_dir = tensorboard_dir+'/'+fname+'_'+label
        pathlib.Path(tb_dir).mkdir(parents=True, exist_ok=True) 
        print('Tensorboard logs in', tb_dir)
        callbacks = [reduce_lr, keras.callbacks.TensorBoard(log_dir=tb_dir, histogram_freq=0)]

    start = time.time()
    hist = model.fit(x_train, Y_train, batch_size=batch_size, epochs=nb_epochs,
              verbose=1, validation_data=(x_test, Y_test), callbacks=callbacks)
    end = time.time()
    log = pd.DataFrame(hist.history)
    
    # Print and save results. Print the testing results which has the lowest training loss.
    label='0'
    print('Training complete on', fname)
    duration_minutes = str(round((end-start)/60))
    print('Training time ', end-start, 'seconds, which is about', duration_minutes, 'minutes.')    
    print('Selected the test result with the lowest training loss. Loss and validation accuracy are -')
    idx = log['loss'].idxmin()
    loss = log.loc[log['loss'].idxmin]['loss']
    val_acc = log.loc[log['loss'].idxmin]['val_acc']
    print(loss, val_acc, 'at index', str(idx), ' (epoch ', str(idx+1), ')')
    summary = '|' + label + '  |'+str(loss)+'  |'+str(val_acc)+' |'+str(idx)+' |'+ duration_minutes + 'mins  |'
    summary_csv = label+','+str(loss)+','+str(val_acc)+','+str(idx)+','+ duration_minutes 
    # Save summary file and log file.
    print('Tensorboard logs in', tb_dir)
    pathlib.Path(logs_dir+'/'+fname).mkdir(parents=True, exist_ok=True) 
    with open(logs_dir+'/'+fname+'/resnet_summary.csv', 'a+') as f:
        f.write(summary_csv)
        f.write('\n')
        print('Added summary row to ', logs_dir+'/'+fname+'/resnet_summary.csv')  
    print('Saving logs to',logs_dir+'/'+fname+'/history_'+label+'.csv')
    log.to_csv(logs_dir+'/'+fname+'/history_'+label+'.csv')

build conv_x
build conv_y
build conv_z
Merging skip connection
build conv_x
build conv_y
build conv_z
Merging skip connection
build conv_x
build conv_y
build conv_z
Merging skip connection
        -- model was built.
Tensorboard logs in ../logs/tensorboard/2018-12-22T09:30/GunPoint_0
Train on 50 samples, validate on 150 samples
Epoch 1/1500
50/50 [==============================] - 3s 54ms/step - loss: 0.6977 - acc: 0.5000 - val_loss: 0.6948 - val_acc: 0.4933
Epoch 2/1500
50/50 [==============================] - 0s 2ms/step - loss: 0.5848 - acc: 0.7200 - val_loss: 0.6943 - val_acc: 0.4933
Epoch 3/1500
50/50 [==============================] - 0s 2ms/step - loss: 1.2138 - acc: 0.5400 - val_loss: 0.6940 - val_acc: 0.4933
Epoch 4/1500
50/50 [==============================] - 0s 2ms/step - loss: 0.5982 - acc: 0.7200 - val_loss: 0.6936 - val_acc: 0.4933
Epoch 5/1500
50/50 [==============================] - 0s 2ms/step - loss: 0.4455 - acc: 0.8400 - val_loss: 0.6933 - val_acc: 0.4933
Epoch 6/1

Epoch 61/1500
50/50 [==============================] - 0s 2ms/step - loss: 0.0014 - acc: 1.0000 - val_loss: 1.1172 - val_acc: 0.5067
Epoch 62/1500
50/50 [==============================] - 0s 2ms/step - loss: 0.0014 - acc: 1.0000 - val_loss: 1.1379 - val_acc: 0.5067
Epoch 63/1500
50/50 [==============================] - 0s 2ms/step - loss: 0.0013 - acc: 1.0000 - val_loss: 1.1587 - val_acc: 0.5067
Epoch 64/1500
50/50 [==============================] - 0s 2ms/step - loss: 0.0013 - acc: 1.0000 - val_loss: 1.1796 - val_acc: 0.5067
Epoch 65/1500
50/50 [==============================] - 0s 2ms/step - loss: 0.0012 - acc: 1.0000 - val_loss: 1.2012 - val_acc: 0.5067
Epoch 66/1500
50/50 [==============================] - 0s 2ms/step - loss: 0.0012 - acc: 1.0000 - val_loss: 1.2234 - val_acc: 0.5067
Epoch 67/1500
50/50 [==============================] - 0s 2ms/step - loss: 0.0012 - acc: 1.0000 - val_loss: 1.2455 - val_acc: 0.5067
Epoch 68/1500
50/50 [==============================] - 0s 2ms/step - 

Epoch 121/1500
50/50 [==============================] - 0s 2ms/step - loss: 3.9741e-04 - acc: 1.0000 - val_loss: 2.6289 - val_acc: 0.5067
Epoch 122/1500
50/50 [==============================] - 0s 2ms/step - loss: 3.9213e-04 - acc: 1.0000 - val_loss: 2.6559 - val_acc: 0.5067
Epoch 123/1500
50/50 [==============================] - 0s 2ms/step - loss: 3.8692e-04 - acc: 1.0000 - val_loss: 2.6829 - val_acc: 0.5067
Epoch 124/1500
50/50 [==============================] - 0s 2ms/step - loss: 3.8188e-04 - acc: 1.0000 - val_loss: 2.7101 - val_acc: 0.5067
Epoch 125/1500
50/50 [==============================] - 0s 2ms/step - loss: 3.7695e-04 - acc: 1.0000 - val_loss: 2.7372 - val_acc: 0.5067
Epoch 126/1500
50/50 [==============================] - 0s 2ms/step - loss: 3.7216e-04 - acc: 1.0000 - val_loss: 2.7642 - val_acc: 0.5067
Epoch 127/1500
50/50 [==============================] - 0s 2ms/step - loss: 3.6742e-04 - acc: 1.0000 - val_loss: 2.7911 - val_acc: 0.5067
Epoch 128/1500
50/50 [============

Epoch 181/1500
50/50 [==============================] - 0s 2ms/step - loss: 2.0633e-04 - acc: 1.0000 - val_loss: 4.1956 - val_acc: 0.5067
Epoch 182/1500
50/50 [==============================] - 0s 2ms/step - loss: 2.0452e-04 - acc: 1.0000 - val_loss: 4.2197 - val_acc: 0.5067
Epoch 183/1500
50/50 [==============================] - 0s 2ms/step - loss: 2.0273e-04 - acc: 1.0000 - val_loss: 4.2436 - val_acc: 0.5067
Epoch 184/1500
50/50 [==============================] - 0s 2ms/step - loss: 2.0098e-04 - acc: 1.0000 - val_loss: 4.2674 - val_acc: 0.5067
Epoch 185/1500
50/50 [==============================] - 0s 2ms/step - loss: 1.9925e-04 - acc: 1.0000 - val_loss: 4.2912 - val_acc: 0.5067
Epoch 186/1500
50/50 [==============================] - 0s 2ms/step - loss: 1.9754e-04 - acc: 1.0000 - val_loss: 4.3148 - val_acc: 0.5067
Epoch 187/1500
50/50 [==============================] - 0s 2ms/step - loss: 1.9586e-04 - acc: 1.0000 - val_loss: 4.3384 - val_acc: 0.5067
Epoch 188/1500
50/50 [============

Epoch 241/1500
50/50 [==============================] - 0s 2ms/step - loss: 1.4883e-04 - acc: 1.0000 - val_loss: 5.3397 - val_acc: 0.5067
Epoch 242/1500
50/50 [==============================] - 0s 2ms/step - loss: 1.4830e-04 - acc: 1.0000 - val_loss: 5.3526 - val_acc: 0.5067
Epoch 243/1500
50/50 [==============================] - 0s 2ms/step - loss: 1.4778e-04 - acc: 1.0000 - val_loss: 5.3653 - val_acc: 0.5067
Epoch 244/1500
50/50 [==============================] - 0s 2ms/step - loss: 1.4727e-04 - acc: 1.0000 - val_loss: 5.3777 - val_acc: 0.5067
Epoch 245/1500
50/50 [==============================] - 0s 2ms/step - loss: 1.4675e-04 - acc: 1.0000 - val_loss: 5.3898 - val_acc: 0.5067
Epoch 246/1500
50/50 [==============================] - 0s 2ms/step - loss: 1.4624e-04 - acc: 1.0000 - val_loss: 5.4017 - val_acc: 0.5067
Epoch 247/1500
50/50 [==============================] - 0s 2ms/step - loss: 1.4572e-04 - acc: 1.0000 - val_loss: 5.4133 - val_acc: 0.5067
Epoch 248/1500
50/50 [============

...

Epoch 1257/1500
50/50 [==============================] - 0s 2ms/step - loss: 6.0091e-05 - acc: 1.0000 - val_loss: 0.0244 - val_acc: 0.9933
Epoch 1258/1500
50/50 [==============================] - 0s 2ms/step - loss: 6.0043e-05 - acc: 1.0000 - val_loss: 0.0244 - val_acc: 0.9933
Epoch 1259/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.9994e-05 - acc: 1.0000 - val_loss: 0.0244 - val_acc: 0.9933
Epoch 1260/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.9944e-05 - acc: 1.0000 - val_loss: 0.0244 - val_acc: 0.9933
Epoch 1261/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.9901e-05 - acc: 1.0000 - val_loss: 0.0244 - val_acc: 0.9933
Epoch 1262/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.9853e-05 - acc: 1.0000 - val_loss: 0.0244 - val_acc: 0.9933
Epoch 1263/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.9806e-05 - acc: 1.0000 - val_loss: 0.0244 - val_acc: 0.9933
Epoch 1264/1500
50/50 [====

Epoch 1316/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.7305e-05 - acc: 1.0000 - val_loss: 0.0243 - val_acc: 0.9933
Epoch 1317/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.7262e-05 - acc: 1.0000 - val_loss: 0.0243 - val_acc: 0.9933
Epoch 1318/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.7218e-05 - acc: 1.0000 - val_loss: 0.0243 - val_acc: 0.9933
Epoch 1319/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.7173e-05 - acc: 1.0000 - val_loss: 0.0243 - val_acc: 0.9933
Epoch 1320/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.7125e-05 - acc: 1.0000 - val_loss: 0.0243 - val_acc: 0.9933
Epoch 1321/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.7083e-05 - acc: 1.0000 - val_loss: 0.0243 - val_acc: 0.9933
Epoch 1322/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.7036e-05 - acc: 1.0000 - val_loss: 0.0243 - val_acc: 0.9933
Epoch 1323/1500
50/50 [====

Epoch 1375/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.4669e-05 - acc: 1.0000 - val_loss: 0.0239 - val_acc: 0.9933
Epoch 1376/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.4630e-05 - acc: 1.0000 - val_loss: 0.0239 - val_acc: 0.9933
Epoch 1377/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.4583e-05 - acc: 1.0000 - val_loss: 0.0239 - val_acc: 0.9933
Epoch 1378/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.4543e-05 - acc: 1.0000 - val_loss: 0.0239 - val_acc: 0.9933
Epoch 1379/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.4496e-05 - acc: 1.0000 - val_loss: 0.0239 - val_acc: 0.9933
Epoch 1380/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.4454e-05 - acc: 1.0000 - val_loss: 0.0239 - val_acc: 0.9933
Epoch 1381/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.4413e-05 - acc: 1.0000 - val_loss: 0.0239 - val_acc: 0.9933
Epoch 1382/1500
50/50 [====

Epoch 1434/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.2169e-05 - acc: 1.0000 - val_loss: 0.0238 - val_acc: 0.9933
Epoch 1435/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.2123e-05 - acc: 1.0000 - val_loss: 0.0238 - val_acc: 0.9933
Epoch 1436/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.2089e-05 - acc: 1.0000 - val_loss: 0.0238 - val_acc: 0.9933
Epoch 1437/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.2045e-05 - acc: 1.0000 - val_loss: 0.0238 - val_acc: 0.9933
Epoch 1438/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.2005e-05 - acc: 1.0000 - val_loss: 0.0238 - val_acc: 0.9933
Epoch 1439/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.1967e-05 - acc: 1.0000 - val_loss: 0.0238 - val_acc: 0.9933
Epoch 1440/1500
50/50 [==============================] - 0s 2ms/step - loss: 5.1924e-05 - acc: 1.0000 - val_loss: 0.0238 - val_acc: 0.9933
Epoch 1441/1500
50/50 [====

Epoch 1493/1500
50/50 [==============================] - 0s 2ms/step - loss: 4.9781e-05 - acc: 1.0000 - val_loss: 0.0237 - val_acc: 0.9933
Epoch 1494/1500
50/50 [==============================] - 0s 2ms/step - loss: 4.9740e-05 - acc: 1.0000 - val_loss: 0.0237 - val_acc: 0.9933
Epoch 1495/1500
50/50 [==============================] - 0s 2ms/step - loss: 4.9700e-05 - acc: 1.0000 - val_loss: 0.0237 - val_acc: 0.9933
Epoch 1496/1500
50/50 [==============================] - 0s 2ms/step - loss: 4.9663e-05 - acc: 1.0000 - val_loss: 0.0237 - val_acc: 0.9933
Epoch 1497/1500
50/50 [==============================] - 0s 2ms/step - loss: 4.9624e-05 - acc: 1.0000 - val_loss: 0.0237 - val_acc: 0.9933
Epoch 1498/1500
50/50 [==============================] - 0s 2ms/step - loss: 4.9586e-05 - acc: 1.0000 - val_loss: 0.0237 - val_acc: 0.9933
Epoch 1499/1500
50/50 [==============================] - 0s 2ms/step - loss: 4.9549e-05 - acc: 1.0000 - val_loss: 0.0237 - val_acc: 0.9933
Epoch 1500/1500
50/50 [====